<a href="https://colab.research.google.com/github/T-Sunm/Learn-pandas/blob/main/Exercise_Indexing%2C_Selecting_%26_Assigning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160616Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D92b2bbe0a4b93d953d07a9467c25f605c6c3a2e7cbf34bee64b642e9a201412789edcfa8c07464cf6d58f58a6fe552ebe5866ee1e3265dd335c4a0ade4c15c975a97305020f283e25455037f4805d9f7da1009f6cd2fed40fc84e2f0ab49a9249e4aa280c70a167d37a7788b095aaf07f6cca3f5631bd110f456e731617016f2e86274d9292b9a1f64d263ff1b60f0e65860584c06cd13f3c76979cb63f520a21b93a9b5bd3e81d33eaaf5c56c7beee93ea548e1ba6d55b090e79dcfae94ef0c60ac25994eda4f85b842a9feccfdb3ac2aabeb3925e42b318f8a23d7c3f616b1e2237ec66344ccc669cd3d573e0601d679d655583067fd6a0bb7ac5a9f1fa4d2,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160616Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D91e2754ef3bc25b5f2ee886ad1c4a385b8b4116f3ded2f98c1b2e98ffa197c9947e71d583aefd06b9a77b7697a58378957a88e03a1d40cabb5a860a25fcd1cc67f2e3ce9cb01caf537a8acec3603a7255d3b3d3504f30bace0a0a2d1e7dcfcc3913528ccc7b15711d99844c8e9f2be8d1471b6a5937862871fc804dbe8a7a4af9938684a7d4c36c9ed80e5a88503456d881106e54b49a6a88e20813fb08a29ae1063d7f7d1417977f87148242787a5223c98b7b12ec3e0008c9393a75d43a1c8a759cd0127b95e235fca9c2d5c07802c52edecab6b0f9f6122e292b0b475ac578ac6aaa701a22c88ccbed5976f843e9f19908ddc2864991ffaa6feaafb27fb99,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160616Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbb37a8e6d3cb261a7ea51e7597d34e7a570d2df598f679e98693ce8d6a8c7bae248fbd517301f17584c38d4462e442364a2d203c0dcf22a9880bd9b3fc7af171214be3befff30d2404bd6b51be7696a5ececb38d7bbc8ddd4129306e496867d3b02360387be554543dff0884880f52614a84364f57cbead7355ec8ea3bfc2ae49071a32d3f600a2db9a3f197b43f8ae49f3e399b9170a9fc612fc313a122d2bbaf243798f43c7134dc634552bd6d9822f98c558a75e7490361a6dc71e552d1f93c40b3d112aa05ab03811d78d988df951ff13305fc73359e7a7aa156382fcb6e33fe86904ee1db00c291051dbe18e4d11a53b9bcca152abe48efa27f765ce9e8,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160616Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8a2f783b08571b90b5f475f023016544c0d310256e041d3b1a909ff92a6727f235bedc57fe9578755b63fff1773b69a501b24b15fc7bc318cbaf561c2416c11955f8e6400720c32e9d89d8a7c9cb0263e2090a2f7bdc5f3efc0b291d5131a5b3eaa8b212655276836d90576251cdb4506bed9f9d00be489e1e23e6dfaeec64e4f29cc0771bd6e77c0bcc4da1d3a3c5ddb769554e4db11df401ffdb983bd5bbb2bb09638205e52cc681a69a3ed036f2366199e33f4ba4dbaf4badc8fa1bc03a5075a43107e355a35e3fa8b91276d49927fef1a67c15677e222f0ad770ae1cfa5ffa88f3bdeeb60b948dcb7e420df2999026305539d26f6a9235fef8754615fca5,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160617Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9203c92a2265cf5af0f1f9a067d619e1d5929e16f8c8812886dfbe60eb1c540ef636de8c0da51010f4e3589bc38cd79594ba03603fcd2b5b4e8fb0bc136a751d57c8987427e1932274fa8ace20887a20dd990b30c321bf56107dfbde38a01e48bfa9aadb062259aa95b904735c0dcf1e26d9edde8f3fba4097c7cc40b181660efcef8d34697882741da92c06e3e99ff48683b58ef5ca0f1f20a89c1f02c42a5a1495d647f2cbe9a83a9eeeef0cc592d0cc42224fe2c1a04d18d638b22f6679c1b863c96e87c77defe5d32d63bab9ddd98a68d3fa2da2583328a349ea0b399f8a6817c4c2eb8f3d102e8ca4d1f88b4d617d7be2a70c29d996fa540cf5bce92cdd,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160617Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2e71f4e34dd008598d51a43f3a4d6a9162a4d194c5447dedbf9fd142445042fe135a3dac1228ba7c70660679e26ab7523d806658f3f47281f4ef9538f4f396023ba810f3247ce57e1f688095ff97d8c52bd466da656647547426cd485ca14fa0503d3d17ada721f37793bb5c52ce48f8e540b3a4d9cb821ae8ec606d587287587190fc8b060d97da9774dc941531faac8e86d353d5335cbd6fa99a63caafc54a9c4fec41aa82f9e134ab86f95e68d692255096da2afadd8fb370de3e99b376a40200a0fc634cac20040a4a3f958c1c685f650c3d4a2d5a8d58eabd36da01db374b46843649427341d8d72d11f378448cf980b9d85a3c27b849d94d931bb32ff0,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160617Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D073302a4513162d2f363ec2823f6c11338f4788c8dce252b5ce9cc398b3bceaac0e29ee01b06ce0eef764eae5f6fbffd5f9813d4f05a45edd12fae8e28a919f368a8cf8b78074aa4e4dbfb22b435bff3613dc2f56f662057acdf0c90ace5925d658f8f8dfab1392c4422b813243f46fe49926a0790741428793dc2f15fa7a077172cb8cb23b9dc7c89bc294e1310f1d53d16790b5b39bbe61c853f5965c8443d7b1ce79bd9a86147a857f4ff0d8b5e4bf4d2f1042a15b47cd9100922721e65566ccc438fe288f375e6ce3af30447fe73f9e6933789cfe6292bfa8154803627e9f62ef64e61f1c4234f862d305a5f3d77ed7c8351922c340f39f57e65e9dbe822,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160617Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D461cd6571c61230c16a5616e3c5b682e68484b10049373ce8e67b0227aee9cc878ce9b67bba1b7d93d0c024837242e0bf5f978164f6f2da5130d5268d7fbf8d62da47e2d0acdf8850b4c0d719d18c60770804001de9eeae70aee9f42a2fccc690e623659b0735cffc12d17ae0172cb4bb0e0459662e08621606aaae517f508afd58235fbfe0cf3746889db731b5e2ac7cf51c5849af87a64651b9c637b6c8ce3380c39e6b2362bf9d005a338632fd896ec9ccf1869e9dae66bf87fbde6f2b34fec744c17430d3404353bdeafbd2da3451641b6bae27c7f97738df522d6d1aa5dd4c5e86c8159ce270535bf35c639167d28bd90bf89c9e97bcc1d9ba6995e1e77,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160617Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D32c1b8325a38b9100ecaa00fdc64d5bc7abe517a349f6f99c9b096c9316ddedc3206e39dd93bb18af55ae2f7b7d9813c925f7a5c1a326d25d4e9770cf9b30ede7c4757f78347d9abb7bdf51c60063e6ec3e252d71dc9fb9b06c6861947fda47aeb1aa9cffaac3efcadef151d8ba96f65649da81af97b8999d1bddac0e205a0ffc1b1856f58334443d0f9d48d9f5022a0355e3d6b09d1deb2653c8c0318ac5eaa83f2c4ceffd67ba02f0f861b1fb1f847805d4a294d61e009824a4e8ce7f6f7e3d301502731861bc47a40091af8016655688ea1844ac13549a8ec7293a3e69c03fef131bad45c73fe99ed202de0d13534f98856900ff8a27ad88b56b2bc0dbc67,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240705%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240705T160617Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D0a542ba418a99b8122b348d51a2726dad94e10e0acb17b0e1cc69b3ccd777e83624edbe0aa17b39877961b47946607cd8084cc3f669c8a08a5800fd195cdcc116b138829ac651d213a32f15b49e919cfa4408b712f47baac04a92859b99986e7343bddcdddac1f8d6e02cef517d36b9b103d15ac0c7d15846489d0fd46071c40f73315d37b24cb7372416c994cd3e8e2acdd8444df3eccd3d11fa18697d373daa1699358876f52081b14aeee98bd749572a712734312b930d2309c1531738f2f394fa9334df4423fb84098551059cdeb11e138a5b176f9375c613964d85b75c8b844bbb0aa47bfdd0edc8e72491fd70f703d144b4a9d3effdeef14bc87f152b8'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/indexing-selecting-assigning).**

---


# Introduction

In this set of exercises we will work with the [Wine Reviews dataset](https://www.kaggle.com/zynicide/wine-reviews).

Run the following cell to load your data and some utility functions (including code to check your answers).

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)
pd.set_option("display.max_rows", 5)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.indexing_selecting_and_assigning import *
print("Setup complete.")

Setup complete.


Look at an overview of your data by running the following line.

In [ ]:
reviews.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


# Exercises

## 1.

Select the `description` column from `reviews` and assign the result to the variable `desc`.

In [ ]:
# Your code here
desc = reviews['description']

# Check your answer
q1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Follow-up question: what type of object is `desc`? If you're not sure, you can check by calling Python's `type` function: `type(desc)`.

In [ ]:
#q1.hint()
#q1.solution()

## 2.

Select the first value from the description column of `reviews`, assigning it to variable `first_description`.

In [ ]:
first_description = reviews.description.iloc[0]
# Check your answer
q2.check()
first_description

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
first_description = reviews.description.iloc[0]
```
Note that while this is the preferred way to obtain the entry in the DataFrame, many other options will return a valid result, such as `reviews.description.loc[0]`, `reviews.description[0]`, and more!  


"Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity."

In [ ]:
#q2.hint()
#q2.solution()

## 3.

Select the first row of data (the first record) from `reviews`, assigning it to the variable `first_row`.

In [ ]:
first_row = reviews.iloc[0]

# Check your answer
q3.check()
first_row

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

country                                                    Italy
description    Aromas include tropical fruit, broom, brimston...
                                     ...                        
variety                                              White Blend
winery                                                   Nicosia
Name: 0, Length: 13, dtype: object

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Select the first 10 values from the `description` column in `reviews`, assigning the result to variable `first_descriptions`.

Hint: format your output as a pandas Series.

In [ ]:
first_descriptions = reviews.loc[:9 , 'description']

# Check your answer
q4.check()
first_descriptions

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
first_descriptions = reviews.description.iloc[:10]
```
Note that many other options will return a valid result, such as `desc.head(10)` and `reviews.loc[:9, "description"]`.    


0    Aromas include tropical fruit, broom, brimston...
1    This is ripe and fruity, a wine that is smooth...
                           ...                        
8    Savory dried thyme notes accent sunnier flavor...
9    This has great depth of flavor with its fresh ...
Name: description, Length: 10, dtype: object

In [ ]:
#q4.hint()
#q4.solution()

## 5.

Select the records with index labels `1`, `2`, `3`, `5`, and `8`, assigning the result to the variable `sample_reviews`.

In other words, generate the following DataFrame:

![](https://storage.googleapis.com/kaggle-media/learn/images/sHZvI1O.png)

In [ ]:
sample_reviews = reviews.iloc[[1, 2, 3, 5, 8],]

# Check your answer
q5.check()
sample_reviews

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
8,Germany,Savory dried thyme notes accent sunnier flavor...,Shine,87,12.0,Rheinhessen,NaN,NaN,Anna Lee C. Iijima,NaN,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,Heinz Eifel


In [ ]:
#q5.hint()
#q5.solution()

## 6.

Create a variable `df` containing the `country`, `province`, `region_1`, and `region_2` columns of the records with the index labels `0`, `1`, `10`, and `100`. In other words, generate the following DataFrame:

![](https://storage.googleapis.com/kaggle-media/learn/images/FUCGiKP.png)

In [ ]:
df = sample_reviews = reviews.loc[[0, 1, 10, 100],["country", "province", "region_1", "region_2"]]

# Check your answer
q6.check()
df

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,province,region_1,region_2
0,Italy,Sicily & Sardinia,Etna,NaN
1,Portugal,Douro,NaN,NaN
10,US,California,Napa Valley,Napa
100,US,New York,Finger Lakes,Finger Lakes


In [ ]:
#q6.hint()
#q6.solution()

## 7.

Create a variable `df` containing the `country` and `variety` columns of the first 100 records.

Hint: you may use `loc` or `iloc`. When working on the answer this question and the several of the ones that follow, keep the following "gotcha" described in the tutorial:

> `iloc` uses the Python stdlib indexing scheme, where the first element of the range is included and the last one excluded.
`loc`, meanwhile, indexes inclusively.

> This is particularly confusing when the DataFrame index is a simple numerical list, e.g. `0,...,1000`. In this case `df.iloc[0:1000]` will return 1000 entries, while `df.loc[0:1000]` return 1001 of them! To get 1000 elements using `loc`, you will need to go one lower and ask for `df.iloc[0:999]`.

In [ ]:
df = sample_reviews = reviews.loc[:99 ,["country", "variety"]]

# Check your answer
q7.check()
df

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct:</span> 


```python
cols = ['country', 'variety']
df = reviews.loc[:99, cols]
```
or 
```python
cols_idx = [0, 11]
df = reviews.iloc[:100, cols_idx]
```


,country,variety
0,Italy,White Blend
1,Portugal,Portuguese Red
...,...,...
98,Italy,Sangiovese
99,US,Bordeaux-style Red Blend


In [ ]:
#q7.hint()
#q7.solution()

## 8.

Create a DataFrame `italian_wines` containing reviews of wines made in `Italy`. Hint: `reviews.country` equals what?

In [ ]:
italian_wines = reviews.loc[reviews.country == 'Italy']

# Check your answer
q8.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
#q8.hint()
#q8.solution()

## 9.

Create a DataFrame `top_oceania_wines` containing all reviews with at least 95 points (out of 100) for wines from Australia or New Zealand.

In [ ]:
top_oceania_wines = reviews.loc[(reviews["points"] >= 95) & ((reviews.country == 'Australia') | (reviews.country == 'New Zealand'))]

# Check your answer
q9.check()
top_oceania_wines

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
345,Australia,This wine contains some material over 100 year...,Rare,100,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscat (Ru...,Muscat,Chambers Rosewood Vineyards
346,Australia,"This deep brown wine smells like a damp, mossy...",Rare,98,350.0,Victoria,Rutherglen,NaN,Joe Czerwinski,@JoeCz,Chambers Rosewood Vineyards NV Rare Muscadelle...,Muscadelle,Chambers Rosewood Vineyards
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122507,New Zealand,"This blend of Cabernet Sauvignon (62.5%), Merl...",SQM Gimblett Gravels Cabernets/Merlot,95,79.0,Hawke's Bay,NaN,NaN,Joe Czerwinski,@JoeCz,Squawking Magpie 2014 SQM Gimblett Gravels Cab...,Bordeaux-style Red Blend,Squawking Magpie
122939,Australia,Full-bodied and plush yet vibrant and imbued w...,The Factor,98,125.0,South Australia,Barossa Valley,NaN,Joe Czerwinski,@JoeCz,Torbreck 2013 The Factor Shiraz (Barossa Valley),Shiraz,Torbreck


In [ ]:
#q9.hint()
#q9.solution()

# Keep going

Move on to learn about **[summary functions and maps](https://www.kaggle.com/residentmario/summary-functions-and-maps)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*